In [2]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [29]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [32]:
# Remove the space for 'FALSE POSITIVE'
new = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[new, "koi_disposition"] = "False_Positive"
df.head(20)["koi_disposition"]

0          CONFIRMED
1     False_Positive
2     False_Positive
3          CONFIRMED
4          CONFIRMED
5          CONFIRMED
6          CONFIRMED
7          CONFIRMED
8          CONFIRMED
9          CONFIRMED
10         CONFIRMED
11    False_Positive
12    False_Positive
13    False_Positive
14         CONFIRMED
15         CONFIRMED
16    False_Positive
17         CONFIRMED
18    False_Positive
19         CONFIRMED
Name: koi_disposition, dtype: object

# Create a Train Test Split

In [33]:
# Asssign the X and y values
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


In [34]:
# Split the data into training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [35]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,...,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,...,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,...,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,...,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,...,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


# Pre-processing

Scale the data using the MinMaxScaler

In [8]:
# Fit data to the MinMax Scaler
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

In [9]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

# Create a Deep Learning Model

In [12]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model = Sequential()
# Add two layers of nodes
model.add(Dense(units=120, activation='relu', input_dim=40))
model.add(Dense(units=120, activation='relu'))
# Add the output layer
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 120)               4920      
_________________________________________________________________
dense_2 (Dense)              (None, 120)               14520     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 363       
Total params: 19,803
Trainable params: 19,803
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Fit the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 2s - loss: 0.5045 - acc: 0.7441
Epoch 2/100
 - 1s - loss: 0.3645 - acc: 0.8095
Epoch 3/100
 - 1s - loss: 0.3509 - acc: 0.8243
Epoch 4/100
 - 1s - loss: 0.3379 - acc: 0.8342
Epoch 5/100
 - 1s - loss: 0.3325 - acc: 0.8368
Epoch 6/100
 - 1s - loss: 0.3233 - acc: 0.8500
Epoch 7/100
 - 1s - loss: 0.3213 - acc: 0.8507
Epoch 8/100
 - 1s - loss: 0.3144 - acc: 0.8518
Epoch 9/100
 - 1s - loss: 0.3158 - acc: 0.8559
Epoch 10/100
 - 1s - loss: 0.3002 - acc: 0.8702
Epoch 11/100
 - 1s - loss: 0.3027 - acc: 0.8632
Epoch 12/100
 - 1s - loss: 0.3053 - acc: 0.8609
Epoch 13/100
 - 1s - loss: 0.2927 - acc: 0.8716
Epoch 14/100
 - 1s - loss: 0.2933 - acc: 0.8719
Epoch 15/100
 - 1s - loss: 0.2893 - acc: 0.8699
Epoch 16/100
 - 1s - loss: 0.2887 - acc: 0.8739
Epoch 17/100
 - 1s - loss: 0.2815 - acc: 0.8763
Epoch 18/100
 - 1s - loss: 0.2855 - acc: 0.8748
Epoch 19/100
 - 1s - loss: 0.2821 - acc: 0.8731
Epoch 20/100
 - 1s - loss: 0.2740 - acc: 0.8803
E

# Quantify Model

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.24534453116255078, Accuracy: 0.8970722781881104


# Make Predictions

In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['FALSE POSITIVE' 'CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'CONFIRMED']
Actual Labels: ['FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED']
